# Modeling language with Bigrams

In [89]:
def read_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

train_raw = "./wikitext-2-raw/wiki.train.txt"
val_raw = "./wikitext-2-raw/wiki.val.txt"
test_raw = "./wikitext-2-raw/wiki.test.txt"

train_text = read_file(train_raw)

chars = sorted(list(set(train_text)))
vocab_size = len(chars)

# Create our character mappings
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

# Create bigrams as simple tuples of indices
bigrams = []
for i in range(len(train_text) - 1):
    input_char = train_text[i]
    target_char = train_text[i + 1]
    bigrams.append([char_to_idx[input_char], char_to_idx[target_char]])

# Let's look at what we've created
print(f"Vocabulary size: {vocab_size}")
print(f"\nFirst 10 characters in vocabulary:")
print(chars[:10])

print(f"\nFirst 5 bigrams as character pairs:")
for i in range(5):
    in_idx, out_idx = bigrams[i]
    print(f"'{idx_to_char[in_idx]}' -> '{idx_to_char[out_idx]}'")

print(f"\nTotal number of bigrams: {len(bigrams)}")

# Optional: let's also see the distribution of characters
from collections import Counter
char_counts = Counter(train_text)
print("\nMost common characters:")
for char, count in char_counts.most_common(10):
    if char.isspace():
        char_display = '<space>'
    elif char == '\n':
        char_display = '<newline>'
    else:
        char_display = char
    print(f"'{char_display}': {count}")

Vocabulary size: 1013

First 10 characters in vocabulary:
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(']

First 5 bigrams as character pairs:
' ' -> '
'
'
' -> ' '
' ' -> '='
'=' -> ' '
' ' -> 'V'

Total number of bigrams: 10918891

Most common characters:
'<space>': 2088677
'e': 990626
't': 690297
'a': 685507
'n': 593094
'i': 588695
'o': 585162
'r': 533179
's': 514154
'h': 387532


## Rationale

We have a text corpus $T$ of length $n$ characters, from this corpus we derive a dataset $\mathbf{X}$ composed of all contiguous bigrams $[x_{i}, x_{i+1}]$ where $i \in 1,\dots,n-1$ (therefore $|X| = n-1$) and $x_{i} \in V$, where $V$ is the set of all distinct characters in $T$ and $|V|=m$. For any character $x_{i}$ the corpus $T$ encodes an underlying, true distribution for $x_{i+1}$ :
$$
P(x_{i+1}|x_{i})
$$
Our objective is to approximate that distribution as best we can by using a model composed of parameters $\Theta$ learned from our dataset $X$. The model will use character $x_{i}$ to predict $x_{i+1}$ ideally we would get:
$$
P(x_{i+1}|x_{i}) \approx P(x_{i+1}|x_{i}, \Theta)\in \mathbb{R}^{m}
$$
More practically, we can say that our model is actually a function $f_{\Theta}$ (parameterized by $\Theta$) where:
$$
\text{softmax}(f_{\Theta}(x_{i})) = P(x_{i+1}|x_{i},\Theta)
$$
Where softmax is used to turn our final $m$ logits into a valid probability distribution.

Now, this is an optimization problem. To be precise, we want to minimize the difference between the real distribution and the distribution we're estimating with out model $f_{\Theta}$. For this, we can pull from the idea of cross-entropy which is the measure that we need:
$$
H(p,q)=-\sum p(x)\log q(x)
$$
Applying it to our problem we would get a specific number for the cross-entropy between the relevant distribution this will be our loss and it will be what we want to minimize:
$$
\mathcal{L} = - \sum_{i=1}^{n-1} P(x_{i+1}|x_{i})\log P(x_{i+1}|x_{i},\Theta)
$$
Obviously, we don't have $P(x_{i+1}|x_{i})$, if we did then problem solved we could just use that, instead all we have is our dataset $X$ which tells us a lot about the real distribution for any one *specific* example.

We have all possible bigrams, therefore for any character $x_{i}$ we know what $x_{x+1}$ actually is, therefore the real distribution $P(x|x_{i})$ has a spike of probability $1$ when $x=x_{i+1}$ and a total of $0$ probability mass when $x=c$, c being any other character other than $x_{i+1}$. In formal terms:
$$
P(x|x_{i}) = \begin{cases}
1  &  \text{if} &  x=x_{i+1} \\
0 & \text{otherwise}
\end{cases}
$$
Since we're conditioning our model probability distribution on predicting $x_{i+1}$ we can say that $P(x_{i+1}|x_{i})=1$ and therefore the loss we have to minimize is:
$$
\mathcal{L} = - \sum_{i=1}^{n-1} \log P(x_{i+1}|x_{i},\Theta)
$$
In practice and mostly for computational reasons we instead calculate the loss over mini batches $B$ where $B \subset X$ (a random selection of training examples), therefore the loss we will actually use is:
$$
\mathcal{L} = -\frac{1}{|B|} \sum_{i \in B} \log (\text{softmax} (f_{\Theta}(x_{i}) )_{y_{i}})
$$
Minimizing this loss amounts to maximizing the probability of seeing the index $y_{i}$ (correct next character according to our data) when out model takes in the character $x_{i}$

## Network y Hand

In [98]:
import torch
import torch.nn.functional as F 

data = torch.tensor(bigrams)
X = data[:, 0]
Y = data[:, 1]

X.shape, Y.shape

(torch.Size([10918891]), torch.Size([10918891]))

In [99]:
embed_dim = 16
hidden_dim = 64

batch_size = 32

embed = torch.randn(vocab_size, embed_dim)
hidden_ff = torch.randn(embed_dim, hidden_dim)
out_ff = torch.randn(hidden_dim, vocab_size)

In [100]:
batch_idx = torch.randint(0, data.shape[0], (batch_size,))
X_batch = X[batch_idx]
X_embed = embed[X_batch]

hidden_logits = X_embed @ hidden_ff
hidden_act = torch.tanh(hidden_logits)

out_logits = hidden_act @ out_ff

probs = F.softmax(out_logits, dim=-1)  

In [ ]:
Y_batch = Y[batch_idx]

# We pluck out the probability of y_i (correct next character) from our probs and compare against the real probability
correct_logprobs = torch.log(probs[torch.arange(batch_size), Y_batch]) 

loss = -correct_logprobs.mean()

loss # Horrible to start

tensor(22.5534)

## Using Torch